## Model building using HydroMT Delft3D FM

This notebook demonstrates how to prepare a **Delft3D FM** model from scratch using the command line interace (CLI).  We will demonstrate the required steps for model building and show the contents of a **configuration file** and the **data catalog**. After following this notebook, you should be able to follow the example notebooks for building the 1D, 2D and coupled 1D2D models.

It is necessary to have a Python environment with HydroMT Core and the HydroMT Delft3D FM plugin installed to be able to run the notebook, as well as additional packages such as **jupyterlab** and **notebook** for notebook functionality, and **cartopy** and **matplotlib** for plotting purposes. The installation guide for HydroMT Delft3D FM can be found in the [User Guide](https://deltares.github.io/hydromt_delft3dfm/latest/getting_started/installation.html).

> **NOTE**: 
All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### Accessing HydroMT through CLI

Lets first check if the Delft3D FM model is recognized by HydroMT using the command `hydromt --models`. This should return **dflowfm**, as well as as well as the generic HydroMT models: **grid_model**, **lumped_model**, **mesh_model**, **network_model**.

> **NOTE**: 
If you have other HydroMT plugins installed in the same environment (Wflow, SFINCS, FIAT, and more), then these models will also appear after this command.

In [1]:
!hydromt --models

model plugins:
 - dflowfm (hydromt_delft3dfm 0.3.0)
generic models (hydromt 0.10.0):
 - grid_model
 - vector_model
 - mesh_model
 - network_model



We can use the command `hydromt --help` to get an overview of all the commands that can be accessed throug the command line interface. HydroMT is typically used to `hydromt build` a model using a configuration file. It is also possible to `hydromt update` or `hydromt clip` existing models. The command `hydromt export` is not directly related to model building, but used to export data from a data catalog. The command `hydromt check` is used to validate the configuration file and the linked data catalogs and region settings. This can especially be useful to catch mistakes in the model building process without having to pass all the model building steps, which can take a lot of time for large models.

A more detailed description of these commands can be found in the [HydroMT API](https://deltares.github.io/hydromt/latest/api/cli.html). 

In [3]:
!hydromt --help

Usage: hydromt [OPTIONS] COMMAND [ARGS]...

  Command line interface for hydromt models.

Options:
  --version  Show the version and exit.
  --models   Print available model plugins and exit.
  --help     Show this message and exit.

Commands:
  build   Build models
  check   Validate config / data catalog / region
  clip    Clip models.
  export  Export data
  update  Update models


> **NOTE**: 
When inspecting the documentation of HydroMT Core, make sure that you open the documentation for the corresponding HydroMT version. The HydroMT version of your Python environment is listed as a part of the `hydromt --models` command, or can be found by simply using `hydromt --version`.

In [4]:
!hydromt --version

HydroMT version: 0.10.0


### Requirements for building a model in HydroMT Delft3D FM

Using the **HydroMT build** API we can setup a complete model from scratch. Let's get an overview of all the available options:

In [5]:
!hydromt build --help

Usage: hydromt build [OPTIONS] MODEL MODEL_ROOT

  Build models from scratch.

  Example usage: --------------

  To build a wflow model for a subbasin using a point coordinates snapped to
  cells with upstream area >= 50 km2 hydromt build wflow /path/to/model_root
  -i /path/to/wflow_config.yml  -r "{'subbasin': [-7.24, 62.09], 'uparea':
  50}" -d deltares_data -d /path/to/data_catalog.yml -v

  To build a sfincs model based on a bbox hydromt build sfincs
  /path/to/model_root  -i /path/to/sfincs_config.yml  -r "{'bbox':
  [4.6891,52.9750,4.9576,53.1994]}"  -d /path/to/data_catalog.yml -v

Options:
  --opt TEXT               Method specific keyword arguments, see the method
                           documentation of the specific model for more
                           information about the arguments.
  -i, --config PATH        Path to hydroMT configuration file, for the model
                           specific implementation.
  -r, --region TEXT        Set the region for which to 

We can see that there are two required arguments for building a model: `MODEL` describing the type of model to build, and `MODEL_ROOT` which is the root path of the model. Additionally, a HydroMT configuration file can be used using either `-i` or `--config`, a data catalog using either `-d` or `--data` and region settings using `-r` or `--region`. Both the configuration file and the data catalog will be explained in greater detail in this notebook.

> **NOTE**: 
In the current version of HydroMT Delft3D FM, no region settings are required when building a model.

### Model setup configuration

The HydroMT configuration file contains the model setup configuration and determines which methods are used to prepare the different components of a Delft3D FM model and in which order and optionally sets non-default arguments for each method. This configuration is passed to HydroMT using `-i <path_to_configuration_file>`. We have prepared several example yml-files which are available in the model repository [examples folder](https://github.com/Deltares/hydromt_delft3dfm/tree/main/examples) and from the [docs (building a model)](https://deltares.github.io/hydromt_delft3dfm/latest/user_guide/dflowfm_build). These example yml-files are also used in this notebook to build the 1D, 2D and coupled 1D2D models.

Each section, before indent, (e.g. setup_rivers_from_dem) corresponds to a model method. A model method is a step inside the model building process. Each method adds a new component to the model, thus building the model step-by-step. All model methods are explained in the [docs (model components)](https://deltares.github.io/hydromt_delft3dfm/latest/user_guide/dflowfm_model_setup.html). The `global` section contains direct model initialisation properties which are required when the empty model is initialized. It typically contains global settings such as the coordinate system of the model.

We will load the default dflowfm build 1D2D yaml file for inspection:

In [6]:
fn_yml = "dflowfm_build_piave.yml"
with open(fn_yml, "r") as f:
    txt = f.read()
print(txt)

global:
  crs: 3857
  network_snap_offset: 25
  openwater_computation_node_distance: 40

setup_rivers_from_dem:
  region: 
    bbox: [12.4331, 46.4661, 12.5212, 46.5369]
  hydrography_fn: merit_hydro
  river_geom_fn: hydro_rivers_lin
  rivers_defaults_fn: rivers_defaults
  rivdph_method: gvf
  rivwth_method: geom
  river_upa: 25.0
  friction_type: "Manning"
  friction_value: 0.023
  rivbankq: 25

setup_pipes:
  region: 
    bbox: [12.4331, 46.4661, 12.5212, 46.5369]
  pipes_fn: grip_roads
  pipes_defaults_fn: pipes_defaults
  pipe_filter: pipe
  spacing: 50
  friction_type: WhiteColeBrook
  friction_value: 0.003
  crosssections_shape: circle
  crosssections_value: 0.5
  dem_fn: merit_hydro # [copdem30, merit]
  pipes_depth: 2.0
  snap_offset: 0.5
  pipes_invlev: 3. 

setup_manholes:
  manholes_fn:
  manholes_defaults_fn: manholes_defaults
  dem_fn: merit_hydro
  bedlevel_shift: 0.5

setup_1dboundary:
  boundary_value: -2.0
  branch_type: river
  boundary_type: waterlevel
  boundary_uni

It can be seen that many arguments are required for the methods. Some arguments require data sources, typically indicated using `<...>_fn`. For example, the `setup_pipes` method requires a data source for the geometry of the pipes: `pipes_fn`, and additionally a data source with the default geometry of these pipes: `pipes_default_fn`. These data sources are obtained either directly by providing a path to the corresponding file, or by using an entry from a given **data catalog**. The next section explains how to create and use a **data catalog**.

> **NOTE**: 
  In this example, we apply region argument in setup_rivers_from_dem, setup_pipes and setup_mesh2d. They are identical in this example but one can alter them to have different region for rivers, for pipes and for 2dmesh. This allows the user to customize their model as much as possible, as the extent of the 1D river can differ from the extent of the 1D urban network or the 2D inundation grid. The region bbox are in WGS 84 as required, but the user should specify the dflowfm model destination CRS in the `global` section in order to allow to use grid definition data (rivers, pipes) from different data sources or CRS. 

### Data for model setup

Most of the time, the `<...>_fn` arguments correspond to a data source from the `DataCatalog` which is based on a yml-file with references to the data paths/urls and how the data should be read. This file can be provided to hydromt using `-d <path_to_yml_file>`. The functionality of the data catalog is prescribed by HydroMT Core and therefore, the detailed description on how to create and use a data catalog can be found in the [HydroMT core docs](https://deltares.github.io/hydromt/latest/user_guide/data_prepare_cat.html). It is possible to use multiple data catalogs by repeating the `-d` flag.

A data catalog in HydroMT is a structured way to organize and reference various datasets, making it easier to manage and access data for modelling. Its entries describe where data can be found, which adapter to use to read data with which settings, how to change the data when reading (scaling, off-setting) and metadata.

> **NOTE**: In HydroMT-Delft3D FM, an additional data catalog with all default parameters values is always used [parameters_data.yml](https://github.com/Deltares/hydromt_delft3dfm/tree/main/hydromt_delft3dfm/data/parameters_data.yml). It contains default values for 1D network elements and structures as well as default 2D landuse parameter mapping tables.

We use the pre-defined `artifact_data` catalog in this example notebook which contains some example data for the Piave basin. This data will be downloaded to `~/.hydromt_data/`. To give an example of a data catalog, we will use `hydromt export` to download and export some entries of the `artifact_data` catalog. This will be done for the same region as the example build models, and the target directory will be the **data** folder, which should already exist in the same folder as this notebook

In [10]:
!hydromt export ./data -s grip_roads -s merit_hydro -s vito_2015 -r "{'bbox': [12.4331, 46.4661, 12.5212, 46.5369]}" -d artifact_data -vvv 

2024-11-05 10:35:33,360 - export - log - DEBUG - Appending log messages to file C:\git\hydromt_delft3dfm\examples\data\hydromt.log.
2024-11-05 10:35:33,360 - export - log - INFO - HydroMT version: 0.10.0
2024-11-05 10:35:33,360 - export - main - INFO - Output dir: C:\git\hydromt_delft3dfm\examples\data


After downloading the data, we can inspect the corresponding data catalog:

In [12]:
dc_yml = "./data/data_catalog.yml"
with open(dc_yml, "r") as f:
    txt = f.read()
print(txt)

meta:
  root: C:\git\hydromt_delft3dfm\examples\data
grip_roads:
  data_type: GeoDataFrame
  version: v4
  path: grip_roads.gpkg
  driver: vector
  filesystem: local
  meta:
    category: road
    paper_doi: 10.1088/1748-9326/aabd42
    paper_ref: Meijer et al, 2018
    source_license: CC0-1.0
    source_url: https://www.globio.info/download-grip-dataset
  crs: 4326
merit_hydro:
  data_type: RasterDataset
  version: '1.0'
  path: merit_hydro/{variable}.tif
  driver: raster
  filesystem: local
  meta:
    category: topography
    paper_doi: 10.1029/2019WR024873
    paper_ref: Yamazaki et al. (2019)
    source_license: CC-BY-NC 4.0 or ODbL 1.0
    source_url: http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro
  crs: 4326
vito_2015:
  data_type: RasterDataset
  version: 2.0.2
  path: vito_2015.tif
  driver: raster
  filesystem: local
  meta:
    category: landuse & landcover
    paper_doi: 10.5281/zenodo.3939038
    paper_ref: Buchhorn et al (2020)
    source_url: https://land.copernicus

### Next steps: Model building notebooks

The basic concepts of accessing HydroMT Delft3D FM using CLI have been explained in this notebook, as well as the contents of a **configuration file** and a **data catalog**. You can now continue to the notebooks where Delft3D FM models will be build using an example configuration file and the artifact data. It is advised to first try building the 1D and 2D models (`build_1dmodel.ipynb` and `build_2dmodel.ipynb`) before moving to the coupled 1D2D model (`build_1d2dmodel.ipynb`).

Additionally, an example notebook is included highlighting the functionality of grid refinement (`update_refine_2dgrid.ipynb`) and how to plot the mesh of a model using Python (`plot_dflowfm_mesh.ipynb`).